<a href="https://colab.research.google.com/github/varunbommagunta/Machine_Learning/blob/main/PyTorch/NeuralStyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [36]:
class VGG(nn.Module):
  def __init__(self):
    super().__init__()
    self.required_features = ['0','5','10','19','28']
    self.model = models.vgg19(pretrained = True).features[:29]
  
  def forward(self,x):
    features = []
    for layer_num,layer in enumerate(self.model):
      x = layer(x)
      if str(layer_num) in self.required_features:
        features.append(x)
    return features

In [37]:
def load_image(img):
  image = Image.open(img)
  image = loader(image).unsqueeze(0)
  return image.to(device)

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
imsize = 500

In [39]:
loader = transforms.Compose(
    [
     transforms.Resize((imsize,imsize)),
     transforms.ToTensor()
    ]
)

In [42]:
original_img = load_image('/content/yoda.jpeg')
style_img = load_image('/content/style.jpg')

In [43]:
generated = original_img.clone().requires_grad_(True)
model = VGG().to(device).eval()

In [44]:
total_steps = 1000
learning_rate = 0.01
alpha = 1
beta = 0.05
optimizer = optim.Adam([generated],lr = learning_rate)
gen = ['a.png','b.png','c.png','d.png','e.png','f.png','g.png','h.png',
       'i.png','j.png','k.png','l.png','m.png']
i = 0

In [ ]:
for step in range(total_steps):
  generated_features = model(generated)
  original_img_features = model(original_img)
  style_features = model(style_img)

  style_loss = original_loss = 0

  for gen_feature,orig_feature,style_feature in zip(generated_features,original_img_features,style_features):
    batch_size,channel,height,width = gen_feature.shape
    original_loss += torch.mean((gen_feature - orig_feature)**2)

    G = gen_feature.view(channel,height*width).mm(gen_feature.view(channel,height*width).t())
    A = style_feature.view(channel, height * width).mm(style_feature.view(channel, height * width).t())
    style_loss += torch.mean((G - A) ** 2)     
  total_loss = alpha*original_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()
  if step%100==0:
    print(total_loss)
    save_image(generated,gen[i])
    i+=1

In [ ]:
l